### 스터디 그룹 1일차

In [ ]:
# 코랩에서 실행할 경우 최신 버전의 사이킷런을 설치합니다.
!pip install --upgrade scikit-learn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import platform
if platform.system() == 'Darwin': #맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': #윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': #리눅스 (구글 콜랩)
        plt.rc('font', family='Malgun Gothic') 
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

In [ ]:
# 1.1 Import 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches

# seaborn setting
sns.set_theme(style='whitegrid')
sns.set_palette("twilight")

In [ ]:
# 1.2 Import Datas
# define data path
# data_path = ".../data/"

# Training and Testing Sets
train = pd.read_csv("train_df.csv", encoding='UTF8')
test = pd.read_csv("test_df.csv", encoding='UTF8')

# Submission
submission = pd.read_csv("sample_submission.csv")

In [ ]:
#블로그와 실제 데이터와의 차이가 있음

display("학습 데이터", train)
print()
print()
display("테스트 데이터", test)
print()
print()
display("제출용 df", submission)

In [ ]:
train.info()

In [ ]:
# int를 str로 변환
train['송하인_격자공간고유번호'] = train['송하인_격자공간고유번호'].apply(str)
train['수하인_격자공간고유번호'] = train['수하인_격자공간고유번호'].apply(str)
# print(type(train['수하인_격자공간고유번호'][0]))

# 데이터 슬라이싱
train['송하인_격자공간고유번호'] = train['송하인_격자공간고유번호'].str.slice(start=0, stop=4)
train['수하인_격자공간고유번호'] = train['수하인_격자공간고유번호'].str.slice(start=0, stop=4)

In [ ]:
train

In [ ]:
# 1.2.2 Testing Set

# int를 str로 변환
test['송하인_격자공간고유번호'] = test['송하인_격자공간고유번호'].apply(str)
test['수하인_격자공간고유번호'] = test['수하인_격자공간고유번호'].apply(str)

# 데이터 슬라이싱
test['송하인_격자공간고유번호'] = test['송하인_격자공간고유번호'].str.slice(start=0, stop=4)
test['수하인_격자공간고유번호'] = test['수하인_격자공간고유번호'].str.slice(start=0, stop=4)

In [ ]:
test

In [ ]:
# 2.1 SEND_SPG_INNB(송하인)

plt.subplot(2, 1, 1)
order_send = train.groupby('송하인_격자공간고유번호').count()['운송장_건수'].sort_values()
order_send_plot = order_send.tail(10).plot(kind='bar', figsize=(30, 10))
order_send_plot.set_xlabel('상위 10 송하인 고유 번호', fontsize=13)
order_send_plot.set_ylabel('운송장 건수', fontsize=13)

plt.subplot(2, 1, 2)
order_send_total = train.groupby('수하인_격자공간고유번호').sum()['운송장_건수'].sort_values()
order_send_total_plot = order_send_total.tail(10).plot(kind='bar', figsize=(30, 10))
order_send_total_plot.set_xlabel('Top 10 SEND_SPG_INNB', fontsize=13)
order_send_total_plot.set_ylabel('Total Number of Ordered Items', fontsize=13)

plt.show()

In [ ]:
print(order_send.tail(10))
print()
print(order_send_total.tail(10))

In [ ]:
# 2.2 REC_SPG_INNB(수하인)

plt.subplot(2, 1, 1)
order_recv = train.groupby('수하인_격자공간고유번호').count()['운송장_건수'].sort_values()
order_recv_plot = order_recv.tail(10).plot(kind='bar', figsize=(30, 10))
order_recv_plot.set_xlabel('Top 10 REC_SPG_INNB', fontsize=13)
order_recv_plot.set_ylabel('Number of Recived Items', fontsize=13)

plt.subplot(2, 1, 2)
order_recv_total = train.groupby('수하인_격자공간고유번호').sum()['운송장_건수'].sort_values()
order_recv_total_plot = order_recv_total.tail(10).plot(kind='bar', figsize=(30, 10))
order_recv_total_plot.set_xlabel('Top 10 REC_SPG_INNB', fontsize=13)
order_recv_total_plot.set_ylabel('Total Number of Recived Items', fontsize=13)

plt.show()

In [ ]:
# 2.3 DL_GD_LCLS_NM(카테고리 대) -> 필요 없을지도

catb_count = train.groupby('물품_카테고리').count()['운송장_건수'].sort_values(ascending=False)
catb_count_total = train.groupby('물품_카테고리').sum()['운송장_건수'].sort_values(ascending=False)

print(catb_count)
print()
print(catb_count_total)

In [ ]:
    1   2
a : 3  2
b : 4  1
c : 3  2

In [ ]:
# 2.4 DL_GD_MCLS_NM(카테고리 중)

# .groupby().count() -> 그룹 내 값의 갯수
# .groupby().sum() -> 그룹 내 값 합계
# sort_values : 값을 기준으로 정렬하는 메소드 / ascending=False는 내림차순
catm_count = train.groupby('물품_카테고리').count()['운송장_건수'].sort_values(ascending=False)
catm_count_total = train.groupby('물품_카테고리').sum()['운송장_건수'].sort_values(ascending=False)

catm_count.rename('주문 건수(갯수)', inplace=True)
catm_count_total.rename('주문량(주문 합계)', inplace=True)

cat_count = pd.concat([catm_count, catm_count_total], axis=1)
cat_count

In [ ]:
# 3.1 SEND_SPG_INNB 평균 값

avg_send = order_send_total/order_send
# print(type(avg_send))
avg_send.rename('avg_send', inplace=True)
# print()
# print(avg_send)
avg_send = avg_send.reset_index().rename(columns={'index': '송하인_격자공간고유번호'})
avg_send

In [ ]:
# train & test 데이터 프레임 병합
train_merge = pd.merge(left = train, right = avg_send, 
                 how = 'left', on = '송하인_격자공간고유번호')
display(train_merge)

test_merge = pd.merge(left = test, right = avg_send, 
                 how = 'left', on = '송하인_격자공간고유번호')
display(test_merge)

train.info()

In [ ]:
test.info()

In [ ]:
# 3.2 REC_SPG_INNB 평균 값

avg_rec = order_recv_total/order_recv
avg_rec.rename('avg_rec', inplace=True)
avg_rec = avg_rec.reset_index().rename(columns={'index': '수하인_격자공간고유번호'})
avg_rec

In [ ]:
# 데이터 프레임 병합(직전에 만든 데이터 프레임에 더하기)
train_merge_rec = pd.merge(left=train_merge, right=avg_rec, 
                 how='left', on=['수하인_격자공간고유번호'])
display(train_merge_rec)
test_merge_rec = pd.merge(left=test_merge, right=avg_rec, 
                 how='left', on=['수하인_격자공간고유번호'])
display(test_merge_rec)

In [ ]:
# 3.3 카테고리 대 평균 값  -> 필요 없을지도

avg_catb = catb_count_total/catb_count
avg_catb.rename('avg_catb', inplace=True)
avg_catb = avg_catb.reset_index().rename(columns={'index': 'DL_GD_LCLS_NM'})

In [ ]:
# 3.4 카테고리 중 평균 값

avg_catm = catm_count_total/catm_count
avg_catm.rename('avg_catm', inplace=True)
avg_catm = avg_catm.reset_index().rename(columns={'index': '물품_카테고리'})

# 데이터 프레임 병합(직전에 만든 데이터 프레임에 더하기)
train_merge_catm = pd.merge(left=train_merge_rec, right=avg_catm,
                 how='left', on=['물품_카테고리'])
display(train_merge_catm)
test_merge_catm = pd.merge(left=test_merge_rec, right=avg_catm, 
                 how='left', on=['물품_카테고리'])
display(test_merge_catm)

In [ ]:
# 4.1 Correlation Matrix

# Compute the correlation matrix
corr = train_merge_catm.drop(['index'], axis=1).corr()

# Generate a mask for the upper triangle
# numpy.triu 함수는 상삼각행렬 (Upper triangular matrix)을 반환합니다.
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# 4.2 Spliting the data sets to X and y

# Training X and y
X = train_merge_catm.drop(['운송장_건수', 'index', '송하인_격자공간고유번호', '수하인_격자공간고유번호',
                '물품_카테고리'], axis=1)
y = train_merge_catm.운송장_건수

# Testing X
X_test = test_merge_catm.drop(['index', '송하인_격자공간고유번호', '수하인_격자공간고유번호',
                    '물품_카테고리'], axis=1)

print(X)
print()
print(y)
print()
print(X_test)